In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader

In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()

running on device cpu


In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr = 0.005, momentum = 0.9, weight_decay = 0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
train = DataLoader(WindowDataset(), batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train))[0])
print(next(iter(train))[1])

(tensor([[[  1.,   1.,   2.,  ...,  65.,  53.,  41.],
         [  1.,   1.,   1.,  ...,  70.,  62.,  48.],
         [  0.,   0.,   0.,  ...,  75.,  72.,  59.],
         ...,
         [125., 127., 121.,  ...,  28.,  27.,  26.],
         [128., 128., 117.,  ...,  28.,  27.,  26.],
         [118., 116., 100.,  ...,  28.,  27.,  26.]],

        [[  1.,   1.,   1.,  ..., 109.,  97.,  85.],
         [  1.,   1.,   1.,  ..., 112., 104.,  90.],
         [  1.,   1.,   1.,  ..., 115., 112.,  99.],
         ...,
         [209., 211., 205.,  ...,  79.,  78.,  77.],
         [212., 212., 199.,  ...,  79.,  78.,  77.],
         [202., 200., 182.,  ...,  79.,  78.,  77.]],

        [[  0.,   0.,   0.,  ...,  96.,  84.,  72.],
         [  0.,   0.,   0.,  ..., 100.,  92.,  78.],
         [  0.,   0.,   0.,  ..., 106., 103.,  90.],
         ...,
         [219., 221., 215.,  ...,  80.,  79.,  78.],
         [222., 222., 210.,  ...,  80.,  79.,  78.],
         [212., 210., 193.,  ...,  80.,  79.,  78.]]

In [11]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [6]:
num_epochs = 40

for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(model, optimizer, train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset

print("DONE")

1 torch.Size([3, 96, 128])
1 {'boxes': tensor([[ 77.,  48., 153., 143.]]), 'labels': tensor([0])}


RuntimeError: expected scalar type Double but found Float